# Score 🙊toxic comments with trained 🤗⚡Detoxify

**Repository:** https://github.com/Borda/detoxify

**Inspiration:** https://www.kaggle.com/sorenj/scoring-comments-using-unitaryai-detoxify

## Prepare environment

In [ ]:
# ! pip install -q detoxify
! pip install -q https://github.com/Borda/detoxify/archive/refs/heads/load-offline.zip

# ! pip download -q detoxify --prefer-binary --dest frozen_packages
! pip wheel -q https://github.com/Borda/detoxify/archive/refs/heads/load-offline.zip --wheel-dir frozen_packages
! rm frozen_packages/torch-*
! ls frozen_packages

In [ ]:
! ls -l /kaggle/input/huggingface-roberta-variants

In [ ]:
! mkdir checkpoints
! wget -q https://github.com/unitaryai/detoxify/releases/download/v0.1-alpha/toxic_original-c1212f89.ckpt --directory-prefix=checkpoints
! wget -q https://github.com/unitaryai/detoxify/releases/download/v0.3-alpha/toxic_debiased-c7548aa0.ckpt --directory-prefix=checkpoints
! wget -q https://github.com/unitaryai/detoxify/releases/download/v0.4-alpha/multilingual_debiased-0b549669.ckpt --directory-prefix=checkpoints
! wget -q https://github.com/unitaryai/detoxify/releases/download/v0.1.2/original-albert-0e1d6498.ckpt --directory-prefix=checkpoints
! wget -q https://github.com/unitaryai/detoxify/releases/download/v0.1.2/unbiased-albert-c8519128.ckpt --directory-prefix=checkpoints
! ls -l checkpoints

In [ ]:
import pandas as pd

comments = pd.read_csv('/kaggle/input/jigsaw-toxic-severity-rating/comments_to_score.csv', index_col='comment_id')
display(comments.head())

## Running predictions

In [ ]:
import torch
loaded = torch.load('checkpoints/toxic_debiased-c7548aa0.ckpt')
loaded["config"]["arch"]["args"]

In [ ]:
from detoxify import Detoxify

model = Detoxify(
    checkpoint='checkpoints/toxic_debiased-c7548aa0.ckpt',
    pretrained_model_path="/kaggle/input/huggingface-roberta-variants/roberta-base/roberta-base",
#     device='cuda',
)
print(comments['text'].tolist()[0])
model.predict(comments['text'].tolist()[0])

In [ ]:
from tqdm import tqdm

tqdm.pandas()
comments['score'] = comments['text'].progress_map(lambda line: model.predict(line)['toxicity'])
comments['score'].plot.hist(bins=50, grid=True)

In [ ]:
comments['score'].to_csv('submission.csv')

! head submission.csv

## Validation

In [ ]:
validation_data = pd.read_csv("/kaggle/input/jigsaw-toxic-severity-rating/validation_data.csv")
display(validation_data.head())

validation_unique = set(validation_data["less_toxic"].tolist() + validation_data["more_toxic"].tolist())
print(f'all: {len(validation_data)} and unique (less_toxic): {len(validation_data["less_toxic"].unique())}')
print(f'all: {len(validation_data)} and unique (more_toxic): {len(validation_data["more_toxic"].unique())}')
print(f'all: {len(validation_data)} and unique (both): {len(validation_unique)}')

In [ ]:
valid_text_score = {txt: model.predict(txt) for txt in tqdm(validation_unique)}

In [ ]:
import numpy as np

pred_fields = ('toxicity', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat', 'sexual_explicit')
for field in pred_fields:
    validation_data['score_less'] = validation_data["less_toxic"].apply(lambda ln: valid_text_score[ln][field])
    validation_data['score_more'] = validation_data["more_toxic"].apply(lambda ln: valid_text_score[ln][field])
    # Average Agreement with Annotators -- score by organizers
    score_less = np.array(validation_data['score_less'])
    score_more = np.array(validation_data['score_more'])
    score_mean = (score_less < score_more).mean()
    print(f"score for {field}: \t{score_mean}")
    validation_data[field] = score_more - score_less

_= validation_data[list(pred_fields)].plot.hist(bins=50, grid=True, alpha=0.4, legend=True)